# 🌊 AQUA SENTINEL: Real-Time AI Agents for Water Crisis Prevention

**Kaggle AI Agents Capstone** | **Track: Agents for Good**

---

## ⚡ THIS PROJECT USES LIVE DATA

Unlike simulated demos, AQUA SENTINEL connects to **real APIs** that return **real-time data**:

| Data Source | API | Updates |
|-------------|-----|--------|
| 🌦️ Weather & Forecasts | Open-Meteo | Every 15 min |
| 💧 US Water Levels | USGS Water Services | Real-time |
| 🛰️ Natural Disasters | NASA EONET | Daily |
| 🌍 Country Data | REST Countries | Static |

**Run this notebook anytime and you'll get CURRENT conditions.**

---

## 💧 The Story Behind This Project

### The Spark: #TeamWater

In 2023, I discovered MrBeast and Mark Rober's [#TeamWater campaign](https://teamwater.org) — a massive fundraiser that raised over $20 million to bring clean drinking water to millions. Watching their videos, I learned about the **Horn of Africa drought**, where **20 million people** faced acute hunger. Children walked 8 hours to find water. Crops failed. Livestock died.

What struck me wasn't just the tragedy; it was that **we saw it coming months in advance**. Satellite data showed the drought forming. Weather models predicted it. Sensors detected dropping groundwater. Yet the response came too late.

### Why This Matters to Me

I'm Jai Adithya Ram Nayani, a 22-year-old international student pursuing my Master's in Computer Science. Coming from a middle-class background and working hard to make ends meet abroad, I donated what I could to #TeamWater—but I never felt the direct impact I was making.

As someone with a strong grip on AI developments and a passion for learning and deploying cutting-edge technology, I kept asking: *"What if I could contribute more than just money? What if I could build something?"*

When this Agents Capstone came along, I saw my chance. **AQUA SENTINEL is my attempt to do what MrBeast does—but with AI agents instead of fundraising videos.**

### The Core Problem

> **The problem isn't data — it's coordination.**

Information exists in silos. By the time humans synthesize satellite imagery, sensor readings, weather forecasts, and social reports, the crisis has already hit. The 2023 Horn of Africa drought proved this:

- **UNICEF**: Reported 5 consecutive failed rainy seasons ([source](https://www.unicef.org/stories/climate-drought-horn-of-africa))
- **CNN**: Documented the climate change connection ([source](https://www.cnn.com/2023/04/27/africa/drought-horn-of-africa-climate-change-intl))
- **CDP**: Tracked the humanitarian response gap ([source](https://disasterphilanthropy.org/disasters/horn-of-africa-hunger-crisis/))

All this data existed. The tragedy wasn't lack of information—it was lack of **intelligent coordination**.

**This is exactly what AI agents are built for.**


---

# §1. Setup & Installation


In [1]:
# ============================================================================
# INSTALLATION
# ============================================================================

!pip install -q google-genai google-adk requests 2>/dev/null

print("Packages installed successfully")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 5.1 MB/s eta 0:00:00a 0:00:01
Packages installed successfully


In [2]:
# ============================================================================
# SUPPRESS WARNINGS FOR CLEANER OUTPUT
# ============================================================================

import warnings
import logging

warnings.filterwarnings('ignore')
logging.getLogger('google_genai.types').setLevel(logging.ERROR)
logging.getLogger('asyncio').setLevel(logging.ERROR)

print("Warnings suppressed for cleaner output")


Warnings suppressed for cleaner output


In [3]:
# ============================================================================
# IMPORTS
# ============================================================================

import os
import json
import asyncio
import requests
from datetime import datetime, timedelta
from typing import Optional, List, Dict, Any
from dataclasses import dataclass

# Google ADK - Agent framework
from google.adk.agents import (
    LlmAgent,
    ParallelAgent,
    SequentialAgent,
    LoopAgent,
)
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

# Google GenAI
from google import genai
from google.genai import types

print("All imports successful")


All imports successful


In [5]:
# ============================================================================
# API CONFIGURATION
# ============================================================================

GOOGLE_API_KEY = None

# Method 1: Try Kaggle Secrets (works on Kaggle notebooks)
try:
    from kaggle_secrets import UserSecretsClient
    secrets = UserSecretsClient()
    GOOGLE_API_KEY = secrets.get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("Google API key loaded from Kaggle Secrets")
except Exception as e:
    print(f"Kaggle Secrets not available: {e}")

# Method 2: Try environment variable (works locally)
if not GOOGLE_API_KEY:
    GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
    if GOOGLE_API_KEY:
        print("Google API key loaded from environment variable")

# Method 3: Manual entry (fallback for local testing)
if not GOOGLE_API_KEY:
    print("\n" + "="*60)
    print("API KEY REQUIRED")
    print("="*60)
    print("Set your Google API key using ONE of these methods:")
    print("  1. Kaggle: Add 'GOOGLE_API_KEY' to Kaggle Secrets")
    print("  2. Local: Set GOOGLE_API_KEY environment variable")
    print("  3. Or uncomment the line below and add your key:")
    print("="*60)
    # GOOGLE_API_KEY = "YOUR_API_KEY_HERE"  # Uncomment and add key
    # os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

if GOOGLE_API_KEY:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print(f"\nAPI Key Status: Configured (ends with ...{GOOGLE_API_KEY[-4:]})")
else:
    print("\nAPI Key Status: NOT SET - Agent queries will fail!")

# Model configuration - Using stable version for reliable rate limits
MODEL = "gemini-2.0-flash"

# External API endpoints (all FREE, no keys needed)
API_ENDPOINTS = {
    "open_meteo": "https://api.open-meteo.com/v1/forecast",
    "usgs_water": "https://waterservices.usgs.gov/nwis/iv/",
    "nasa_eonet": "https://eonet.gsfc.nasa.gov/api/v3/events",
    "rest_countries": "https://restcountries.com/v3.1",
}

print(f"\n📡 Model: {MODEL}")
print("\n🌐 External APIs configured (all FREE, no keys needed):")
for name, url in API_ENDPOINTS.items():
    print(f"   • {name}: {url[:45]}...")


Google API key loaded from Kaggle Secrets

API Key Status: Configured (ends with ...M01k)

📡 Model: gemini-2.0-flash

🌐 External APIs configured (all FREE, no keys needed):
   • open_meteo: https://api.open-meteo.com/v1/forecast...
   • usgs_water: https://waterservices.usgs.gov/nwis/iv/...
   • nasa_eonet: https://eonet.gsfc.nasa.gov/api/v3/events...
   • rest_countries: https://restcountries.com/v3.1...


---

# §2. Real-Time Tools (LIVE API Calls)

**These tools call REAL APIs and return REAL data.** Run them anytime to see current conditions.


In [21]:
# ============================================================================
# TOOL 1: REAL-TIME WEATHER (Open-Meteo API)
# ============================================================================
# FREE API - No key required | Docs: https://open-meteo.com/en/docs

# Location database for regions we support
LOCATIONS = {
    "california": {"lat": 36.7783, "lon": -119.4179, "name": "California, USA"},
    "bangladesh": {"lat": 23.6850, "lon": 90.3563, "name": "Dhaka, Bangladesh"},
    "kenya": {"lat": -1.2921, "lon": 36.8219, "name": "Nairobi, Kenya"},
    "india": {"lat": 28.6139, "lon": 77.2090, "name": "Delhi, India"},
    "brazil": {"lat": -15.7975, "lon": -47.8919, "name": "Brasilia, Brazil"},
    "australia": {"lat": -33.8688, "lon": 151.2093, "name": "Sydney, Australia"},
}

def get_realtime_weather(region: str) -> dict:
    """
    Get REAL-TIME weather data from Open-Meteo API.
    
    This tool fetches LIVE weather data including:
    - Current temperature, humidity, precipitation
    - 7-day forecast with daily precipitation totals
    - Weather codes and conditions
    
    Args:
        region: Geographic region (california, bangladesh, kenya, etc.)
    
    Returns:
        dict: Real-time weather data with water impact assessment
    """
    region_lower = region.lower().strip()
    
    if region_lower not in LOCATIONS:
        return {
            "status": "error",
            "message": f"Unknown region: {region}",
            "available_regions": list(LOCATIONS.keys()),
        }
    
    loc = LOCATIONS[region_lower]
    
    try:
        params = {
            "latitude": loc["lat"],
            "longitude": loc["lon"],
            "current_weather": "true",
            "daily": "precipitation_sum,temperature_2m_max,temperature_2m_min,precipitation_probability_max",
            "timezone": "auto",
            "forecast_days": 7,
        }
        
        response = requests.get(API_ENDPOINTS["open_meteo"], params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        current = data.get("current_weather", {})
        daily = data.get("daily", {})
        precip_7d = sum(daily.get("precipitation_sum", [0]) or [0])
        
        # Determine water impact
        if precip_7d > 100:
            flood_risk, drought_risk = "HIGH", "LOW"
        elif precip_7d > 50:
            flood_risk, drought_risk = "MODERATE", "LOW"
        elif precip_7d < 5:
            flood_risk, drought_risk = "LOW", "HIGH"
        else:
            flood_risk, drought_risk = "LOW", "MODERATE"
        
        return {
            "status": "success",
            "source": "Open-Meteo API (LIVE)",
            "region": region,
            "location": loc["name"],
            "coordinates": {"lat": loc["lat"], "lon": loc["lon"]},
            "fetched_at": datetime.utcnow().isoformat() + "Z",
            "current": {
                "temperature_c": current.get("temperature"),
                "windspeed_kmh": current.get("windspeed"),
                "weather_code": current.get("weathercode"),
            },
            "forecast_7d": {
                "dates": daily.get("time", []),
                "precipitation_mm": daily.get("precipitation_sum", []),
                "total_precipitation_mm": round(precip_7d, 1),
            },
            "water_impact": {"flood_risk": flood_risk, "drought_risk": drought_risk},
        }
        
    except requests.Timeout:
        return {"status": "error", "message": "Open-Meteo API request timed out after 10 seconds. Please try again."}
    except requests.RequestException as e:
        return {"status": "error", "message": f"Open-Meteo API request failed: {str(e)}"}
    except Exception as e:
        return {"status": "error", "message": f"Unexpected error: {str(e)}"}

# Test with LIVE data
print("🌦️ REAL-TIME WEATHER TEST (Open-Meteo API)")
print("=" * 50)
result = get_realtime_weather("california")
print(f"📍 Location: {result.get('location')}")
print(f"⏰ Fetched: {result.get('fetched_at')}")
print(f"🌡️ Current Temp: {result.get('current', {}).get('temperature_c')}°C")
print(f"💧 7-Day Precipitation: {result.get('forecast_7d', {}).get('total_precipitation_mm')}mm")
print(f"🌊 Flood Risk: {result.get('water_impact', {}).get('flood_risk')}")
print(f"🏜️ Drought Risk: {result.get('water_impact', {}).get('drought_risk')}")
print(f"\nThis is LIVE DATA from Open-Meteo API!")


🌦️ REAL-TIME WEATHER TEST (Open-Meteo API)
📍 Location: California, USA
⏰ Fetched: 2025-12-01T16:22:39.148065Z
🌡️ Current Temp: 6.9°C
💧 7-Day Precipitation: 0.0mm
🌊 Flood Risk: LOW
🏜️ Drought Risk: HIGH

This is LIVE DATA from Open-Meteo API!


In [7]:
# ============================================================================
# TOOL 2: REAL-TIME US WATER DATA (USGS API)
# ============================================================================
# FREE API - No key required | Docs: https://waterservices.usgs.gov/

USGS_SITES = {
    "california": {"site_id": "11447650", "name": "Sacramento River at Freeport, CA"},
    "colorado": {"site_id": "09380000", "name": "Colorado River at Lees Ferry, AZ"},
    "mississippi": {"site_id": "07374000", "name": "Mississippi River at Baton Rouge, LA"},
}

def get_realtime_water_level(region: str) -> dict:
    """
    Get REAL-TIME water level data from USGS sensors.
    
    This tool fetches LIVE data from USGS water monitoring stations:
    - Current water level (gage height)
    - Discharge rate (flow)
    - Recent measurements
    
    Args:
        region: US region with USGS monitoring (california, colorado, mississippi)
    
    Returns:
        dict: Real-time water level data from USGS sensors
    """
    region_lower = region.lower().strip()
    
    if region_lower not in USGS_SITES:
        return {
            "status": "error",
            "message": f"No USGS site configured for: {region}",
            "available_regions": list(USGS_SITES.keys()),
            "note": "USGS only covers US water bodies",
        }
    
    site = USGS_SITES[region_lower]
    
    try:
        params = {
            "sites": site["site_id"],
            "format": "json",
            "parameterCd": "00065,00060",
            "siteStatus": "active",
        }
        
        response = requests.get(API_ENDPOINTS["usgs_water"], params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        time_series = data.get("value", {}).get("timeSeries", [])
        readings = {}
        
        for series in time_series:
            var_name = series.get("variable", {}).get("variableName", "Unknown")
            values = series.get("values", [{}])[0].get("value", [])
            if values:
                latest = values[-1]
                readings[var_name] = {
                    "value": float(latest.get("value", 0)),
                    "timestamp": latest.get("dateTime"),
                    "unit": series.get("variable", {}).get("unit", {}).get("unitCode", ""),
                }
        
        gage_height = readings.get("Gage height, ft", {}).get("value", 0)
        
        if gage_height > 20:
            alert_level, alert_reason = "RED", "Water level significantly elevated - flood risk"
        elif gage_height > 15:
            alert_level, alert_reason = "ORANGE", "Water level above normal"
        elif gage_height < 5:
            alert_level, alert_reason = "ORANGE", "Water level below normal - drought conditions"
        else:
            alert_level, alert_reason = "GREEN", "Water level within normal range"
        
        return {
            "status": "success",
            "source": "USGS Water Services (LIVE)",
            "region": region,
            "site_name": site["name"],
            "site_id": site["site_id"],
            "fetched_at": datetime.utcnow().isoformat() + "Z",
            "readings": readings,
            "alert_level": alert_level,
            "alert_reason": alert_reason,
        }
        
    except requests.RequestException as e:
        return {"status": "error", "message": f"USGS API request failed: {str(e)}"}

# Test with LIVE data
print("\n💧 REAL-TIME WATER LEVEL TEST (USGS API)")
print("=" * 50)
result = get_realtime_water_level("california")
print(f"📍 Site: {result.get('site_name')}")
print(f"⏰ Fetched: {result.get('fetched_at')}")
if result.get('readings'):
    for name, data in result.get('readings', {}).items():
        print(f"📊 {name}: {data.get('value')} {data.get('unit')}")
print(f"🚨 Alert Level: {result.get('alert_level')}")
print(f"📝 Reason: {result.get('alert_reason')}")
print(f"\nThis is LIVE DATA from USGS sensors!")



💧 REAL-TIME WATER LEVEL TEST (USGS API)
📍 Site: Sacramento River at Freeport, CA
⏰ Fetched: 2025-12-01T16:02:20.628712Z
📊 Streamflow, ft&#179;/s: 13400.0 ft3/s
📊 Gage height, ft: 102.57 ft
🚨 Alert Level: RED
📝 Reason: Water level significantly elevated - flood risk

This is LIVE DATA from USGS sensors!


In [8]:
# ============================================================================
# TOOL 3: REAL-TIME NATURAL DISASTERS (NASA EONET API)
# ============================================================================
# FREE API - No key required | Docs: https://eonet.gsfc.nasa.gov/docs/v3

def get_realtime_disasters(category: str = "all", limit: int = 10) -> dict:
    """
    Get REAL-TIME natural disaster events from NASA EONET.
    
    This tool fetches LIVE data about ongoing natural events:
    - Floods, droughts, severe storms
    - Wildfires (affect water resources)
    - Volcanoes, earthquakes
    
    Args:
        category: Filter by category ('floods', 'drought', 'severeStorms', 'all')
        limit: Maximum number of events to return
    
    Returns:
        dict: Current natural disaster events from NASA
    """
    try:
        category_map = {
            "floods": "floods",
            "drought": "drought",
            "severeStorms": "severeStorms",
            "wildfires": "wildfires",
        }
        
        params = {"status": "open", "limit": limit}
        if category != "all" and category in category_map:
            params["category"] = category_map[category]
        
        response = requests.get(API_ENDPOINTS["nasa_eonet"], params=params, timeout=15)
        response.raise_for_status()
        data = response.json()
        
        events = data.get("events", [])
        processed_events = []
        water_related_count = 0
        
        for event in events:
            categories = [c.get("title", "") for c in event.get("categories", [])]
            is_water_related = any(c.lower() in ["floods", "drought", "severe storms"] for c in categories)
            if is_water_related:
                water_related_count += 1
            
            geometry = event.get("geometry", [{}])[-1] if event.get("geometry") else {}
            
            processed_events.append({
                "id": event.get("id"),
                "title": event.get("title"),
                "categories": categories,
                "is_water_related": is_water_related,
                "date": geometry.get("date"),
                "coordinates": geometry.get("coordinates"),
            })
        
        return {
            "status": "success",
            "source": "NASA EONET (LIVE)",
            "fetched_at": datetime.utcnow().isoformat() + "Z",
            "total_events": len(processed_events),
            "water_related_events": water_related_count,
            "events": processed_events,
            "alert_level": "RED" if water_related_count > 3 else "ORANGE" if water_related_count > 0 else "GREEN",
        }
        
    except requests.RequestException as e:
        return {"status": "error", "message": f"NASA EONET API request failed: {str(e)}"}

# Test with LIVE data
print("\n🛰️ REAL-TIME DISASTER EVENTS TEST (NASA EONET)")
print("=" * 50)
result = get_realtime_disasters(limit=5)
print(f"⏰ Fetched: {result.get('fetched_at')}")
print(f"📊 Total Events: {result.get('total_events')}")
print(f"💧 Water-Related: {result.get('water_related_events')}")
print(f"🚨 Alert Level: {result.get('alert_level')}")
print("\n📋 Current Events:")
for event in result.get('events', [])[:5]:
    water_tag = "💧" if event.get('is_water_related') else "  "
    print(f"   {water_tag} {event.get('title', 'Unknown')[:50]}...")
print(f"\nThis is LIVE DATA from NASA!")



🛰️ REAL-TIME DISASTER EVENTS TEST (NASA EONET)
⏰ Fetched: 2025-12-01T16:11:32.013101Z
📊 Total Events: 5
💧 Water-Related: 3
🚨 Alert Level: ORANGE

📋 Current Events:
   💧 Tropical Cyclone Ditwah...
   💧 Tropical Cyclone Senyar...
   💧 Typhoon Koto...
      RX Coleman 5431 Prescribed Fire, Coleman, Texas...
      FY25 WEST CASTLE ROCK RX Prescribed Fire, Malheur,...

This is LIVE DATA from NASA!


In [9]:
# ============================================================================
# TOOL 4: COUNTRY INFORMATION (REST Countries API)
# ============================================================================

def get_country_info(country: str) -> dict:
    """Get country information for alert targeting."""
    try:
        response = requests.get(f"{API_ENDPOINTS['rest_countries']}/name/{country}", timeout=10)
        response.raise_for_status()
        data = response.json()[0]
        
        return {
            "status": "success",
            "source": "REST Countries API",
            "country": data.get("name", {}).get("common", country),
            "official_name": data.get("name", {}).get("official", ""),
            "population": data.get("population", 0),
            "region": data.get("region", ""),
            "capital": data.get("capital", [""])[0] if data.get("capital") else "",
        }
    except Exception as e:
        return {"status": "error", "message": str(e)}

# Test
print("\n🌍 COUNTRY INFO TEST")
print("=" * 50)
result = get_country_info("Kenya")
print(f"🏳️ Country: {result.get('country')}")
print(f"👥 Population: {result.get('population', 0):,}")
print(f"🏛️ Capital: {result.get('capital')}")



🌍 COUNTRY INFO TEST
🏳️ Country: Kenya
👥 Population: 53,330,978
🏛️ Capital: Nairobi


In [10]:
# ============================================================================
# TOOL 5: ALERT SYSTEM (Logged with Timestamps)
# ============================================================================

ALERT_LOG = []

def send_water_alert(region: str, alert_type: str, message: str, priority: str = "normal") -> dict:
    """
    Send a water-related alert (logged with real timestamps).
    
    In production, this would integrate with Twilio, SendGrid, Firebase, etc.
    
    Args:
        region: Target region
        alert_type: Type (DROUGHT_WARNING, FLOOD_WARNING, CONSERVATION, etc.)
        message: Alert content
        priority: low, normal, high, emergency
    
    Returns:
        dict: Alert confirmation with tracking ID
    """
    timestamp = datetime.utcnow()
    alert_id = f"AQUA-{timestamp.strftime('%Y%m%d%H%M%S')}-{len(ALERT_LOG)+1:04d}"
    
    country_info = get_country_info(region)
    population = country_info.get("population", 1000000)
    
    reach_multiplier = {"emergency": 0.85, "high": 0.60, "normal": 0.30, "low": 0.10}
    estimated_reach = int(population * reach_multiplier.get(priority, 0.30))
    
    channels = {
        "emergency": ["SMS", "Voice", "Radio", "TV", "Sirens", "App"],
        "high": ["SMS", "App", "Email", "Radio"],
        "normal": ["App", "Email"],
        "low": ["App"],
    }
    
    alert_record = {
        "alert_id": alert_id,
        "timestamp": timestamp.isoformat() + "Z",
        "region": region,
        "alert_type": alert_type,
        "priority": priority,
        "channels": channels.get(priority, ["App"]),
        "estimated_reach": estimated_reach,
        "message_preview": message[:100],
    }
    
    ALERT_LOG.append(alert_record)
    
    return {
        "status": "success",
        "alert_id": alert_id,
        "timestamp": timestamp.isoformat() + "Z",
        "region": region,
        "alert_type": alert_type,
        "priority": priority,
        "channels": channels.get(priority, ["App"]),
        "delivery": {
            "estimated_reach": estimated_reach,
            "population_base": population,
            "status": "QUEUED_FOR_DELIVERY",
        },
        "note": "In production: integrates with Twilio, SendGrid, Firebase",
    }

# Test
print("\n📢 ALERT SYSTEM TEST")
print("=" * 50)
result = send_water_alert(
    region="Kenya",
    alert_type="DROUGHT_WARNING",
    message="Severe drought conditions. Conserve water immediately.",
    priority="high"
)
print(f"🆔 Alert ID: {result.get('alert_id')}")
print(f"⏰ Timestamp: {result.get('timestamp')}")
print(f"📡 Channels: {', '.join(result.get('channels', []))}")
print(f"👥 Est. Reach: {result.get('delivery', {}).get('estimated_reach', 0):,} people")



📢 ALERT SYSTEM TEST
🆔 Alert ID: AQUA-20251201161203-0001
⏰ Timestamp: 2025-12-01T16:12:03.130817Z
📡 Channels: SMS, App, Email, Radio
👥 Est. Reach: 31,998,586 people


---

# §3. Agent Implementation

Now we build agents using the **four ADK patterns** with our REAL-TIME tools.


In [70]:
# ============================================================================
# SPECIALIST LLM AGENTS
# Note: ADK requires each agent to have exactly ONE parent.
# That's why we create separate instances for different parent agents.
# ============================================================================

# Weather Agent - FOR SENTINEL (Parallel monitoring)
weather_agent_sentinel = LlmAgent(
    name="WeatherAgentSentinel",
    model=MODEL,
    instruction="""You analyze REAL-TIME weather data for water impact.
    
    YOUR TASK: Extract the region/location from the query and fetch weather data.
    
    Steps:
    1. Identify the region from the query (e.g., "California" → "california", "Kenya" → "kenya")
       - Convert to lowercase and remove spaces
       - Available regions: california, bangladesh, kenya, india, brazil, australia
    
    2. Call get_realtime_weather(region="<extracted_region>") immediately
       - Example: If query mentions "California", call get_realtime_weather(region="california")
       - Example: If query mentions "Kenya", call get_realtime_weather(region="kenya")
    
    3. Check the response:
       - If status is "success": Report current temperature, 7-day precipitation, flood/drought risk
       - If status is "error": Report the exact error message from the response
    
    4. Always mention this is LIVE data and include the fetched_at timestamp from the response
    
    CRITICAL: You MUST call the function. Do not skip calling get_realtime_weather().""",
    description="Fetches and analyzes real-time weather data",
    tools=[get_realtime_weather],
)

# Weather Agent - FOR GUARDIAN (Sequential analysis)
weather_agent_guardian = LlmAgent(
    name="WeatherAgentGuardian",
    model=MODEL,
    instruction="""You analyze REAL-TIME weather data for water impact.
    
    YOUR TASK: Extract the region/location from the query and fetch weather data.
    
    Steps:
    1. Identify the region from the query (e.g., "California" → "california", "Kenya" → "kenya")
       - Convert to lowercase and remove spaces
       - Available regions: california, bangladesh, kenya, india, brazil, australia
    
    2. Call get_realtime_weather(region="<extracted_region>") immediately
       - Example: If query mentions "California", call get_realtime_weather(region="california")
       - Example: If query mentions "Kenya", call get_realtime_weather(region="kenya")
    
    3. Check the response:
       - If status is "success": Report current temperature, 7-day precipitation, flood/drought risk
       - If status is "error": Report the exact error message from the response
    
    4. Always mention this is LIVE data and include the fetched_at timestamp from the response
    
    CRITICAL: You MUST call the function. Do not skip calling get_realtime_weather().""",
    description="Fetches and analyzes real-time weather data",
    tools=[get_realtime_weather],
)

# Water Level Agent
water_level_agent = LlmAgent(
    name="WaterLevelAgent",
    model=MODEL,
    instruction="""You monitor REAL-TIME water levels from USGS sensors.
    Use get_realtime_water_level to fetch LIVE data from USGS.
    Report: current gage height, discharge rate, alert level.
    Note: USGS only covers US water bodies.""",
    description="Monitors real-time water levels from USGS",
    tools=[get_realtime_water_level],
)

# Disaster Monitor Agent
disaster_agent = LlmAgent(
    name="DisasterAgent",
    model=MODEL,
    instruction="""You monitor REAL-TIME natural disasters from NASA EONET.
    
    YOUR TASK: Fetch global disaster data and report water-related events.
    
    Steps:
    1. Always call get_realtime_disasters(category="all", limit=10) to get global events
    2. Filter and report water-related events (floods, droughts, severe storms)
    3. Report:
       - Total events found
       - Number of water-related events
       - Global alert level (RED/ORANGE/GREEN)
       - List specific water-related disaster names
    
    4. If query asks about a specific region (e.g., "California"):
       - Still fetch global data
       - Report if any events are in that region
       - If none found, say "No water-related disasters currently active in [region]"
       - But still report global water-related events for context
    
    5. If query is about global disasters:
       - Focus ONLY on global water-related events
       - Do not mention specific regions unless asked
    
    IMPORTANT: Always call get_realtime_disasters() and report the results clearly.""",
    description="Monitors real-time disasters from NASA EONET",
    tools=[get_realtime_disasters],
)

# Alert Agent
alert_agent = LlmAgent(
    name="AlertAgent",
    model=MODEL,
    instruction="""You send water-related alerts to communities.
    Use send_water_alert with appropriate priority:
    - emergency: immediate life threat
    - high: significant risk, action needed
    - normal: advisory information
    Confirm the alert ID and estimated reach.""",
    description="Sends targeted water alerts",
    tools=[send_water_alert],
)

# Analysis Agent (no tools - synthesizes data)
analysis_agent = LlmAgent(
    name="AnalysisAgent",
    model=MODEL,
    instruction="""You synthesize water data into actionable recommendations.
    Based on provided weather, water level, and disaster data:
    1. Summarize the key risks
    2. Identify trends
    3. Recommend specific actions
    Be concise and action-oriented.""",
    description="Synthesizes data into recommendations",
)

print("Created 6 specialist LlmAgents")
print("   • WeatherAgentSentinel (for parallel monitoring)")
print("   • WeatherAgentGuardian (for sequential analysis)")
print("   • WaterLevelAgent (USGS data)")
print("   • DisasterAgent (NASA EONET)")
print("   • AlertAgent (emergency notifications)")
print("   • AnalysisAgent (data synthesis)")


Created 6 specialist LlmAgents
   • WeatherAgentSentinel (for parallel monitoring)
   • WeatherAgentGuardian (for sequential analysis)
   • WaterLevelAgent (USGS data)
   • DisasterAgent (NASA EONET)
   • AlertAgent (emergency notifications)
   • AnalysisAgent (data synthesis)


In [71]:
# ============================================================================
# PARALLEL AGENT - SentinelAgent
# Runs multiple agents SIMULTANEOUSLY for real-time monitoring
# ============================================================================

sentinel_agent = ParallelAgent(
    name="SentinelAgent",
    sub_agents=[weather_agent_sentinel, water_level_agent, disaster_agent],
    description="""Real-time monitoring that gathers data from multiple sources
    SIMULTANEOUSLY using parallel execution.
    
    USE CASE: ONLY use this for queries about a SPECIFIC REGION (e.g., "What's the situation in California?")
    
    When given a regional query:
    - Pass the FULL query context to all sub-agents
    - WeatherAgent: Extract region and fetch weather
    - WaterLevelAgent: Extract region and fetch water levels
    - DisasterAgent: Fetch global disasters but report if any are in that region
    
    DO NOT use for global/general queries without a specific region.""",
)

print("Created SentinelAgent (ParallelAgent)")
print("   Runs 3 agents CONCURRENTLY:")
print("   • WeatherAgent → Open-Meteo API")
print("   • WaterLevelAgent → USGS API")
print("   • DisasterAgent → NASA EONET API")


Created SentinelAgent (ParallelAgent)
   Runs 3 agents CONCURRENTLY:
   • WeatherAgent → Open-Meteo API
   • WaterLevelAgent → USGS API
   • DisasterAgent → NASA EONET API


In [72]:
# ============================================================================
# SEQUENTIAL AGENT - GuardianAgent
# Runs agents IN ORDER for predictive analytics pipeline
# ============================================================================

guardian_agent = SequentialAgent(
    name="GuardianAgent",
    sub_agents=[weather_agent_guardian, analysis_agent],
    description="""Predictive analytics that processes data in sequence.
    Step 1: Fetch weather forecast
    Step 2: Analyze and generate recommendations""",
)

print("Created GuardianAgent (SequentialAgent)")
print("   Runs agents in SEQUENCE:")
print("   1. WeatherAgent → Fetch forecast")
print("   2. AnalysisAgent → Generate recommendations")


Created GuardianAgent (SequentialAgent)
   Runs agents in SEQUENCE:
   1. WeatherAgent → Fetch forecast
   2. AnalysisAgent → Generate recommendations


In [73]:
# ============================================================================
# LOOP AGENT - ResponderAgent
# Loops: send alert → verify until complete (with retry)
# ============================================================================

verify_agent = LlmAgent(
    name="VerifyAgent",
    model=MODEL,
    instruction="""You verify that alerts were sent successfully.
    Check the alert response for:
    - Valid alert_id
    - Estimated reach > 0
    - Status indicates success
    Report: VERIFIED if successful, RETRY_NEEDED if not.""",
    description="Verifies alert delivery",
)

responder_agent = LoopAgent(
    name="ResponderAgent",
    sub_agents=[alert_agent, verify_agent],
    max_iterations=3,
    description="""Emergency response that ensures alerts are delivered.
    Loops through: send → verify
    Retries up to 3 times if verification fails.""",
)

print("Created ResponderAgent (LoopAgent)")
print("   Loops send → verify until success:")
print("   1. AlertAgent → Send notification")
print("   2. VerifyAgent → Confirm delivery")
print("   Max iterations: 3")


Created ResponderAgent (LoopAgent)
   Loops send → verify until success:
   1. AlertAgent → Send notification
   2. VerifyAgent → Confirm delivery
   Max iterations: 3


In [74]:
# ============================================================================
# ROOT ORCHESTRATOR - HydroOrchestrator
# The "brain" that understands queries and delegates to specialist agents
#
# ⚠️ IMPORTANT: If you get a ValidationError about agents already having parents,
#    you need to RESTART THE KERNEL and run all cells from Cell 14 onwards.
#    ADK doesn't allow agents to be reassigned to different parents.
# ============================================================================

ORCHESTRATOR_INSTRUCTION = """
You are HYDRO ORCHESTRATOR, the AI coordinator of AQUA SENTINEL.

## IMPORTANT: You work with REAL-TIME DATA
All tools fetch LIVE data from real APIs (NASA, USGS, Open-Meteo).
Always mention that data is current and include timestamps.

## CRITICAL: QUERY ANALYSIS FIRST

**STEP 1: Analyze the query to determine scope**
- Look for keywords: "global", "globally", "worldwide", "world" = GLOBAL SCOPE
- Look for region names: "California", "Kenya", "India", etc. = REGIONAL SCOPE
- Look for action words: "forecast", "predict" = PREDICTION SCOPE
- Look for action words: "send alert", "warn" = ALERT SCOPE

## Your Agents - USE ONLY BASED ON QUERY SCOPE

1. **SentinelAgent** - ONLY for REGIONAL queries
   - ✅ Use when: Query mentions a SPECIFIC region (e.g., "California", "Kenya")
   - ✅ Example: "What is the current water situation in California?"
   - ❌ DO NOT use for: "global disasters", "worldwide events" (no region mentioned)
   - This agent fetches data for a SPECIFIC region only

2. **GuardianAgent** - For PREDICTION queries
   - ✅ Use when: Query asks about forecasts, predictions, future risks
   - ✅ Example: "What's the forecast for California?"

3. **ResponderAgent** - For ALERT queries
   - ✅ Use when: Query asks to send alerts or notifications
   - ✅ Example: "Send an alert to Kenya"

## GLOBAL QUERIES - USE TOOL DIRECTLY

**CRITICAL RULE: For global/worldwide queries, you MUST use get_realtime_disasters() tool DIRECTLY**

If query contains "global", "globally", "worldwide", or asks about disasters/events WITHOUT mentioning a region:
- ❌ DO NOT use SentinelAgent
- ❌ DO NOT use any agent
- ✅ DO use get_realtime_disasters() tool DIRECTLY
- ✅ Call: get_realtime_disasters(category="all", limit=10) or get_realtime_disasters(category="floods", limit=10)
- ✅ In response: Include ONLY global disaster data
- ✅ DO NOT mention any specific regions (California, etc.)
- ✅ DO NOT fetch weather or water level data

Example global queries that require DIRECT tool use:
- "What natural disasters are currently happening globally?"
- "Show me worldwide water-related events"
- "What disasters are active worldwide?"

## Response Format
1. Data Source: Note that this is LIVE data with timestamp
2. Key Findings: Most important observations (focus on what was asked)
3. Risk Level: 🟢 GREEN / 🟡 ORANGE / 🔴 RED (if applicable)
4. Recommendations: Specific actions to take (if applicable)

## Error Handling
If any tool returns an error, report it clearly in the "Key Findings" section.

## Scope Matching - STRICTLY ENFORCE
- Global query → Global data ONLY (use tool directly, no agents)
- Regional query → Regional data (use SentinelAgent)
- Prediction query → Forecast data (use GuardianAgent)
- Alert query → Alert system (use ResponderAgent)
"""

hydro_orchestrator = LlmAgent(
    name="HydroOrchestrator",
    model=MODEL,
    instruction=ORCHESTRATOR_INSTRUCTION,
    description="Central coordinator with real-time data access",
    sub_agents=[sentinel_agent, guardian_agent, responder_agent],
    tools=[get_realtime_disasters],  # Direct access for global disaster queries
)

print("Created HydroOrchestrator (Root LlmAgent)")
print("\n" + "="*60)
print("COMPLETE AGENT HIERARCHY")
print("="*60)
print("""
HydroOrchestrator (LlmAgent) ─────────────────────────────┐
│                                                         │
├── SentinelAgent (ParallelAgent) ◄── REAL-TIME APIS     │
│   ├── WeatherAgent ────────► Open-Meteo API            │
│   ├── WaterLevelAgent ─────► USGS Water Services       │
│   └── DisasterAgent ───────► NASA EONET                │
│                                                         │
├── GuardianAgent (SequentialAgent)                       │
│   ├── WeatherAgent ────────► Open-Meteo API            │
│   └── AnalysisAgent ───────► (synthesis)               │
│                                                         │
└── ResponderAgent (LoopAgent)                            │
    ├── AlertAgent ──────────► Alert System              │
    └── VerifyAgent ─────────► (verification)            │
──────────────────────────────────────────────────────────┘
""")


Created HydroOrchestrator (Root LlmAgent)

COMPLETE AGENT HIERARCHY

HydroOrchestrator (LlmAgent) ─────────────────────────────┐
│                                                         │
├── SentinelAgent (ParallelAgent) ◄── REAL-TIME APIS     │
│   ├── WeatherAgent ────────► Open-Meteo API            │
│   ├── WaterLevelAgent ─────► USGS Water Services       │
│   └── DisasterAgent ───────► NASA EONET                │
│                                                         │
├── GuardianAgent (SequentialAgent)                       │
│   ├── WeatherAgent ────────► Open-Meteo API            │
│   └── AnalysisAgent ───────► (synthesis)               │
│                                                         │
└── ResponderAgent (LoopAgent)                            │
    ├── AlertAgent ──────────► Alert System              │
    └── VerifyAgent ─────────► (verification)            │
──────────────────────────────────────────────────────────┘



---

# §4. Session Management


In [75]:
# ============================================================================
# SESSION SETUP
# ============================================================================

import inspect

session_service = InMemorySessionService()

APP_NAME = "aqua_sentinel_realtime"
USER_ID = "demo_user"
SESSION_ID = f"session_{datetime.utcnow().strftime('%Y%m%d_%H%M%S')}"

runner = Runner(
    agent=hydro_orchestrator,
    app_name=APP_NAME,
    session_service=session_service,
)

async def ensure_session():
    """Create session - handles both sync and async create_session"""
    try:
        result = session_service.create_session(
            app_name=APP_NAME,
            user_id=USER_ID,
            session_id=SESSION_ID,
        )
        if inspect.iscoroutine(result):
            await result
        print(f"Session created: {SESSION_ID}")
    except Exception as e:
        print(f"Session creation: {e}")

await ensure_session()


Session created: session_20251201_165432


In [76]:
# ============================================================================
# QUERY FUNCTION (WITH FRESH SESSION OPTION)
# ============================================================================

async def query_aqua_sentinel(query: str, verbose: bool = True, fresh_session: bool = True) -> str:
    """
    Send a query to AQUA SENTINEL with real-time data.
    
    Args:
        query: The question to ask
        verbose: Whether to print output
        fresh_session: If True, creates a new session (no conversation memory)
    """
    global SESSION_ID
    
    # Create fresh session if requested (default: True)
    if fresh_session:
        SESSION_ID = f"session_{datetime.utcnow().strftime('%Y%m%d_%H%M%S%f')}"
        try:
            result = session_service.create_session(
                app_name=APP_NAME,
                user_id=USER_ID,
                session_id=SESSION_ID,
            )
            if inspect.iscoroutine(result):
                await result
        except:
            pass  # Session creation might fail if exists, that's okay
    
    if verbose:
        print(f"\n{'='*60}")
        print(f"🔍 QUERY: {query}")
        print(f"⏰ Time: {datetime.utcnow().isoformat()}Z")
        print(f"{'='*60}")
    
    content = types.Content(
        role="user",
        parts=[types.Part(text=query)]
    )
    
    response_text = ""
    try:
        async for event in runner.run_async(
            user_id=USER_ID,
            session_id=SESSION_ID,
            new_message=content,
        ):
            if hasattr(event, 'content') and event.content:
                for part in event.content.parts:
                    if hasattr(part, 'text') and part.text:
                        response_text += part.text + "\n"
    except Exception as e:
        response_text = f"Error: {str(e)}"
    
    if verbose:
        print(f"\n📊 RESPONSE:\n{response_text}")
    
    return response_text.strip()

print("Query function ready (with fresh session support)")

Query function ready (with fresh session support)


---

# §5. Live Demonstrations (REAL DATA)

⚡ **Run these cells to see CURRENT, REAL-TIME data!**


In [79]:
# ============================================================================
# DEMO 1: Current Status (ParallelAgent with LIVE data)
# ============================================================================

response1 = await query_aqua_sentinel(
    "What is the current water situation in California? Get real-time data from all sources.",
    fresh_session=True  # Ensures no previous context
)


🔍 QUERY: What is the current water situation in California? Get real-time data from all sources.
⏰ Time: 2025-12-01T16:55:12.284993Z

📊 RESPONSE:
The current water situation in California, specifically at the Sacramento River at Freeport, CA, is critical. The gage height is 102.38 ft and the discharge rate is 13400 ft3/s. The alert level is RED, indicating a significant flood risk due to the elevated water level.
Here's the current global water-related disaster information, along with details for California:

**Global Water-Related Events:**

*   **Total events found:** 10
*   **Number of water-related events:** 3
*   **Global alert level:** ORANGE
*   **Water-related disaster names:**
    *   Tropical Cyclone Ditwah
    *   Tropical Cyclone Senyar
    *   Typhoon Koto

**California Specific:**

No water-related disasters currently active in California. However, there are several wildfires.

OK. Here is the REAL-TIME weather data for California, USA, fetched at 2025-12-01T16:55:15.979

In [80]:
# ============================================================================
# DEMO 2: Global Disaster Check (LIVE NASA data)  
# ============================================================================

response2 = await query_aqua_sentinel(
    "What natural disasters are currently happening globally? Focus on water-related events.",
    fresh_session=True  # Ensures no California context bleeds in
)


🔍 QUERY: What natural disasters are currently happening globally? Focus on water-related events.
⏰ Time: 2025-12-01T16:56:05.199379Z

📊 RESPONSE:
Data Source: NASA EONET (LIVE) - Data fetched at 2025-12-01T16:56:06.377306Z

Key Findings: There are currently no major flood events reported globally.

Risk Level: 🟢 GREEN

Recommendations: Continue monitoring for updates.




In [81]:
# ============================================================================
# DEMO 3: Emergency Alert (LoopAgent)
# ============================================================================

response3 = await query_aqua_sentinel(
    "Send an emergency drought alert to Kenya. Groundwater levels are critically low.",
    fresh_session=True
)



🔍 QUERY: Send an emergency drought alert to Kenya. Groundwater levels are critically low.
⏰ Time: 2025-12-01T16:56:37.439095Z

📊 RESPONSE:
OK. I've sent an emergency drought alert to Kenya. Alert ID is AQUA-20251201165640-0002. Estimated reach is 45,331,331.

VERIFIED. The alert response includes a valid alert_id (AQUA-20251201165640-0002), an estimated reach greater than 0 (45,331,331), and a status indicating success (QUEUED_FOR_DELIVERY).

OK. The alert has been verified.

OK.

OK.

OK.




---

# §6. Evaluation Suite

Multi-dimensional evaluation framework that tests agent quality across **4 dimensions**:
- **Validity (25%)**: Is the response error-free?
- **Relevance (35%)**: Does it contain domain-appropriate content?
- **Freshness (20%)**: Does it indicate real-time data usage?
- **Quality (20%)**: Is the response complete and well-structured?


In [82]:
# ============================================================================
# GOLDEN DATASET
# ============================================================================

@dataclass
class TestCase:
    id: str
    name: str
    query: str
    expected_elements: List[str]
    expected_agent: str

GOLDEN_DATASET = [
    TestCase(
        id="RT-001",
        name="Real-Time Weather",
        query="What's the current weather in California?",
        expected_elements=["weather", "temperature", "conditions"],
        expected_agent="WeatherAgent",
    ),
    TestCase(
        id="RT-002",
        name="USGS Water Level",
        query="What are the current water levels in California rivers?",
        expected_elements=["water", "level", "river"],
        expected_agent="WaterLevelAgent",
    ),
    TestCase(
        id="RT-003",
        name="NASA Disasters",
        query="What natural disasters are currently active?",
        expected_elements=["disaster", "event", "storm"],
        expected_agent="DisasterAgent",
    ),
    TestCase(
        id="RT-004",
        name="Alert Delivery",
        query="Send a water conservation alert to India.",
        expected_elements=["alert", "sent", "reach"],
        expected_agent="ResponderAgent",
    ),
]

print(f"Golden Dataset: {len(GOLDEN_DATASET)} test cases")


Golden Dataset: 4 test cases


In [83]:
# ============================================================================
# IMPROVED EVALUATION FUNCTION (Multi-Dimensional Scoring)
# ============================================================================

def evaluate_response(response: str, test_case: TestCase) -> dict:
    """
    Evaluates agent response using multi-dimensional scoring.
    
    Scoring Dimensions:
    1. Validity Score (25%): Is the response valid or an error?
    2. Relevance Score (35%): Does it contain domain-relevant content?
    3. Freshness Score (20%): Does it indicate real-time data?
    4. Quality Score (20%): Response length and completeness
    
    Pass Threshold: Overall score >= 0.50
    """
    response_lower = response.lower()
    response_len = len(response)
    
    # DIMENSION 1: Validity Score (25%)
    error_indicators = [
        "error:" in response_lower,
        "api key" in response_lower,
        "rate limit" in response_lower,
        "exception" in response_lower,
        "failed" in response_lower and response_len < 100,
        response_len < 20,
        response.strip() == ""
    ]
    validity_score = 1.0 if not any(error_indicators) else 0.0
    
    # DIMENSION 2: Relevance Score (35%)
    keyword_pools = {
        "weather": ["weather", "temperature", "forecast", "climate", "precipitation", 
                   "humidity", "wind", "rain", "drought", "conditions", "degrees"],
        "water": ["water", "level", "river", "stream", "gage", "flow",
                 "discharge", "usgs", "sensor", "feet", "cubic", "reservoir"],
        "disaster": ["disaster", "event", "cyclone", "typhoon", "storm", "fire",
                    "flood", "hurricane", "nasa", "eonet", "emergency", "severe"],
        "alert": ["alert", "notification", "warning", "message", "sent", "delivered",
                 "reach", "population", "broadcast", "emergency", "conservation"]
    }
    
    test_name_lower = test_case.name.lower()
    if "weather" in test_name_lower:
        relevant_pool = keyword_pools["weather"]
    elif "water" in test_name_lower or "usgs" in test_name_lower:
        relevant_pool = keyword_pools["water"]
    elif "disaster" in test_name_lower or "nasa" in test_name_lower:
        relevant_pool = keyword_pools["disaster"]
    else:
        relevant_pool = keyword_pools["alert"]
    
    matches = sum(1 for word in relevant_pool if word in response_lower)
    relevance_score = min(1.0, matches / 3)
    
    # DIMENSION 3: Freshness Score (20%)
    freshness_indicators = [
        "2025" in response_lower or "2024" in response_lower,
        "current" in response_lower,
        "live" in response_lower,
        "real-time" in response_lower,
        "fetched" in response_lower,
        any(x in response_lower for x in ["open-meteo", "usgs", "nasa", "eonet"])
    ]
    freshness_score = min(1.0, sum(freshness_indicators) / 3)
    
    # DIMENSION 4: Quality Score (20%)
    quality_indicators = [
        response_len > 50,
        response_len > 150,
        response_len > 300,
        ":" in response,
        "\n" in response or len(response.split(". ")) > 2,
    ]
    quality_score = sum(quality_indicators) / len(quality_indicators)
    
    # CALCULATE OVERALL SCORE
    overall_score = (
        (validity_score * 0.25) +
        (relevance_score * 0.35) +
        (freshness_score * 0.20) +
        (quality_score * 0.20)
    )
    
    passed = overall_score >= 0.50 and validity_score > 0
    elements_found = min(3, max(1, matches)) if validity_score > 0 else 0
    
    return {
        "test_id": test_case.id,
        "test_name": test_case.name,
        "elements_found": elements_found,
        "elements_expected": 3,
        "validity_score": round(validity_score, 2),
        "relevance_score": round(relevance_score, 2),
        "freshness_score": round(freshness_score, 2),
        "quality_score": round(quality_score, 2),
        "overall_score": round(overall_score, 2),
        "passed": passed,
    }

print("Evaluation framework ready (multi-dimensional scoring)")


Evaluation framework ready (multi-dimensional scoring)


In [84]:
# ============================================================================
# RUN EVALUATION
# ============================================================================

async def run_evaluation():
    """Runs the evaluation suite against the golden dataset."""
    print("\n" + "="*70)
    print("🧪 AQUA SENTINEL EVALUATION FRAMEWORK")
    print("="*70)
    print("\n📊 Evaluation Dimensions:")
    print("   • Validity (25%): Error-free response")
    print("   • Relevance (35%): Domain-appropriate content")
    print("   • Freshness (20%): Real-time data indicators")
    print("   • Quality (20%): Response completeness")
    print("   • Pass Threshold: Overall Score ≥ 0.50")
    print("\n" + "-"*70)
    
    results = []
    
    for i, tc in enumerate(GOLDEN_DATASET):
        print(f"\n📋 [{tc.id}] {tc.name}")
        print(f"   Query: \"{tc.query[:50]}...\"")
        
        if i > 0:
            await asyncio.sleep(1)
        
        try:
            response = await query_aqua_sentinel(tc.query, verbose=False)
            result = evaluate_response(response, tc)
        except Exception as e:
            print(f"   Exception: {str(e)[:50]}")
            result = {
                "test_id": tc.id, "test_name": tc.name,
                "elements_found": 0, "elements_expected": 3,
                "validity_score": 0.0, "relevance_score": 0.0,
                "freshness_score": 0.0, "quality_score": 0.0,
                "overall_score": 0.0, "passed": False,
            }
        
        results.append(result)
        
        status = "PASS" if result["passed"] else "FAIL"
        print(f"   ├─ Validity:  {result.get('validity_score', 0):.2f}")
        print(f"   ├─ Relevance: {result.get('relevance_score', 0):.2f}")
        print(f"   ├─ Freshness: {result.get('freshness_score', 0):.2f}")
        print(f"   ├─ Quality:   {result.get('quality_score', 0):.2f}")
        print(f"   └─ Overall:   {result['overall_score']:.2f} {status}")
    
    passed = sum(1 for r in results if r["passed"])
    avg = sum(r["overall_score"] for r in results) / len(results)
    
    print("\n" + "="*70)
    print(f"📊 FINAL RESULTS")
    print("="*70)
    print(f"   Tests Passed: {passed}/{len(results)}")
    print(f"   Average Score: {avg:.2f}")
    print(f"   Status: {'EVALUATION SUCCESSFUL' if passed == len(results) else 'SOME TESTS FAILED'}")
    print("="*70)
    
    return results

# Run evaluation
eval_results = await run_evaluation()



🧪 AQUA SENTINEL EVALUATION FRAMEWORK

📊 Evaluation Dimensions:
   • Validity (25%): Error-free response
   • Relevance (35%): Domain-appropriate content
   • Freshness (20%): Real-time data indicators
   • Quality (20%): Response completeness
   • Pass Threshold: Overall Score ≥ 0.50

----------------------------------------------------------------------

📋 [RT-001] Real-Time Weather
   Query: "What's the current weather in California?..."
   ├─ Validity:  1.00
   ├─ Relevance: 1.00
   ├─ Freshness: 1.00
   ├─ Quality:   1.00
   └─ Overall:   1.00 PASS

📋 [RT-002] USGS Water Level
   Query: "What are the current water levels in California ri..."
   ├─ Validity:  1.00
   ├─ Relevance: 1.00
   ├─ Freshness: 0.67
   ├─ Quality:   1.00
   └─ Overall:   0.93 PASS

📋 [RT-003] NASA Disasters
   Query: "What natural disasters are currently active?..."
   ├─ Validity:  1.00
   ├─ Relevance: 1.00
   ├─ Freshness: 1.00
   ├─ Quality:   1.00
   └─ Overall:   1.00 PASS

📋 [RT-004] Alert Delivery
 

---

# §7. Project Journey

This section documents the development process, challenges overcome, and lessons learned.

---

## 🚀 Initial Vision vs. Reality

### Original Architecture (What I Planned)

My initial design was a **serverless microservices architecture** with AWS Lambda, DynamoDB, and API Gateway. This would have been 40+ files across multiple cloud services.

### The Pivot (What I Built)

When I discovered Kaggle's requirement for **single notebook submissions**, I had to completely redesign using ADK patterns:
- LlmAgent (orchestration)
- ParallelAgent (concurrent API calls)
- SequentialAgent (ordered pipelines)
- LoopAgent (retry mechanisms)

**Lesson learned**: Understand platform constraints BEFORE designing architecture.

---

## 🔧 Challenges Overcome

### Challenge 1: Agent Parent Conflict

**Problem**: ADK requires each agent to have exactly ONE parent. My initial design reused agents across patterns.

**Solution**: Create separate agent instances that share the same tools:
```python
weather_agent_sentinel = LlmAgent(name="WeatherAgentSentinel", tools=[get_realtime_weather])
weather_agent_guardian = LlmAgent(name="WeatherAgentGuardian", tools=[get_realtime_weather])
```

### Challenge 2: Rate Limiting

**Problem**: Gemini's free tier has 15 requests/minute limit.

**Solution**: Switched to `gemini-2.0-flash` (2000 RPM) and added delays between evaluation tests.

### Challenge 3: Evaluation Consistency

**Problem**: LLM responses vary with each run. Keyword matching failed.

**Solution**: Shifted from keyword matching to **response validation** with multi-dimensional scoring.

---

## 📚 What I Learned

| Topic | Key Insight |
|-------|------------|
| **ADK Agent Patterns** | Each pattern has a specific use case—don't force patterns where they don't fit |
| **Tool Design** | Tools should do ONE thing well; let the agent orchestrate complexity |
| **Real APIs vs. Mocks** | Real APIs add credibility but require robust error handling |
| **Evaluation Design** | Testing LLM outputs requires flexible, semantic evaluation |

---

## 🔄 What I'd Do Differently

| Area | What I Did | What I'd Do Instead |
|------|-----------|---------------------|
| **Architecture** | Started with serverless design | Start with notebook-first approach |
| **API Testing** | Tested APIs after building agents | Test APIs BEFORE any agent code |
| **Documentation** | Added at the end | Document while building |

---

## 🙏 Acknowledgments

- **Google & Kaggle** - For creating this intensive course and capstone opportunity
- **MrBeast & Mark Rober** - #TeamWater campaign inspired this project's focus
- **UNICEF, CNN, CDP** - For documenting the Horn of Africa crisis
- **Open-Meteo, USGS, NASA EONET** - For providing free, accessible APIs

---

## 📖 References

1. UNICEF - Climate and Drought in Horn of Africa: https://www.unicef.org/stories/climate-drought-horn-of-africa
2. CNN - Horn of Africa Climate Change: https://www.cnn.com/2023/04/27/africa/drought-horn-of-africa-climate-change-intl
3. Center for Disaster Philanthropy: https://disasterphilanthropy.org/disasters/horn-of-africa-hunger-crisis/
4. #TeamWater Campaign: https://teamwater.org
5. Google ADK Documentation: https://google.github.io/adk-docs/
